In [22]:
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import preprocessor as p
import os
import time
from datetime import datetime

In [8]:
!pip3 install preprocessor

  Created wheel for preprocessor: filename=preprocessor-1.1.3-py3-none-any.whl size=4475 sha256=b59862cda2d1c7367b106686f8e518e6859342390f3df7e42781dab6fc8fffec
  Stored in directory: /Users/gauribaraskar/Library/Caches/pip/wheels/e4/4e/bf/0ecf68aa10ee89d684d90437bd9f89ac19d5dc2921988bb59d
Successfully built preprocessor


In [4]:
# Twitter credentials
# Obtain them from your twitter developer account
consumer_key = "fUKXD6HbgzvEOTBi4mIrAfWnL"
consumer_secret = "G6IasvguxSpUgA4GhPPgjr5VeMtGN7bMMIGYMOYyH1em8gWYm9"
access_key = "217978535-7Ces0VMy4neyTXmPyvG2iMe5UZdlLnBKhL3Zqqnc"
access_secret = "TszZdJfRQhZbQ7Rsu1tarClV9e11IWDBggHpoA0heLdEJ"
# Pass your twitter credentials to tweepy via its OAuthHandler
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [23]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [43]:
def scraptweets(search_words, date_since, numTweets, numRuns):
    
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try T times
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['username', 'location', 'profileurl','isdefault',
                                        'totaltweets', 'usercreatedts', 'tweetcreatedts']
                                )
    program_start = time.time()
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        tweets = tweepy.Cursor(api.search_tweets, q='*', lang="en", tweet_mode='extended').items(numTweets)
# Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]
# Obtain the following info (methods to call them out):
        # user.screen_name - twitter handle
        # user.description - description of account
        # user.location - where is he tweeting from
        # user.friends_count - no. of other users that user is following (following)
        # user.followers_count - no. of other users who are following this user (followers)
        # user.statuses_count - total tweets by user
        # user.created_at - when the user account was created
        # created_at - when the tweet was created
        # retweet_count - no. of retweets
        # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
        # retweeted_status.full_text - full text of the tweet
        # tweet.entities['hashtags'] - hashtags in the tweet
# Begin scraping the tweets individually:
        noTweets = 0
        for tweet in tweet_list:
        # Pull the values
            username = tweet.user.screen_name
            #acctdesc = tweet.user.description
            location = tweet.user.location
            profileurl = tweet.user.profile_image_url_https
            isdefault = tweet.user.default_profile_image
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
#             retweetcount = tweet.retweet_count
#             hashtags = tweet.entities['hashtags']
    # Add the 11 variables to the empty list - ith_tweet:
            ith_tweet = [username, location, profileurl, isdefault,totaltweets,
                         usercreatedts, tweetcreatedts]
# Append to dataframe - db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet
# increase counter - noTweets  
            noTweets += 1

                # Run ended:
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)

        print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
        print('time take for {} run to complete is {} mins'.format(i+1, duration_run))

        time.sleep(920) #15 minute sleep time
# Once all runs have completed, save them to a single csv file:
    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
    # Define working path and filename
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_sahkprotests_tweets.csv'
    # Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)

    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))

In [44]:
# Initialise these variables:
#search_words = "#hongkong OR #hkprotests OR #freehongkong OR #hongkongprotests OR #hkpolicebrutality OR #antichinazi OR #standwithhongkong OR #hkpolicestate OR #HKpoliceterrorist OR #standwithhk OR #hkpoliceterrorism"
date_since = "2019-11-03"
numTweets = 2500
numRuns = 50
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)



Rate limit reached. Sleeping for: 841


no. of tweets scraped for run 1 is 2500
time take for 1 run to complete is 15.22 mins


Rate limit reached. Sleeping for: 842


no. of tweets scraped for run 2 is 2500
time take for 2 run to complete is 15.18 mins


Rate limit reached. Sleeping for: 846


...

no. of tweets scraped for run 48 is 2500
time take for 48 run to complete is 17.99 mins


Rate limit reached. Sleeping for: 852


no. of tweets scraped for run 49 is 2500
time take for 49 run to complete is 17.55 mins


Rate limit reached. Sleeping for: 852


no. of tweets scraped for run 50 is 2500
time take for 50 run to complete is 17.67 mins
Scraping has completed!
Total time taken to scrap is 1586.35 minutes.


In [1]:
import pandas as pd

In [56]:
data = pd.read_csv('~/Desktop/merged_age_gender_race.csv')

In [17]:
no_age_users = data.isna().sum(axis=0)

In [20]:
data['screen.name'].unique().shape

(172268,)

In [23]:
data = data.dropna()

In [29]:
data.columns

Index(['Unnamed: 0', 'age', 'name', 'screen.name', 'pred_gender'], dtype='object')

In [31]:
data = data.drop(data.columns[[0, 2]], axis=1)

In [37]:
data.age.max()

74.0

In [59]:
data['age_bins'] = pd.cut(data.age,bins = [14,24,34,44,54,64,75])
data.head()

,Unnamed: 0,age,name,screen.name,pred_gender,age_bins
age_bins,,,,,,
"(14, 24]",5821,5821,5821,5821,5805,NaN
"(24, 34]",39650,39650,39649,39649,39562,NaN
"(34, 44]",38255,38255,38254,38254,38214,NaN
"(44, 54]",8982,8982,8982,8982,8976,NaN
"(54, 64]",2008,2008,2008,2008,2008,NaN


In [58]:
 data = data.groupby(['age_bins']).count()

In [62]:
data['age_percent'] = data['age'].div(85807).round(2)

In [63]:
labels = '(14-24]', '', 'Dogs', 'Logs'
sizes = [15, 30, 45, 10]
explode = (0, 0.1, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

1.1

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd


for username in usernames:
    
    print(username)
# Creating list to append tweet data to
    tweets_list1 = []

    query = f'from:{username}'
    print(query)
    # Using TwitterSearchScraper to scrape data and append tweets to list
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
        if i>1000:
            break
        tweets_list1.append([tweet.user.username, tweet.date, tweet.id,tweet.user.location])

    # Creating a dataframe from the tweets list above 
    tweets_df1 = pd.DataFrame(tweets_list1, columns=['Username','Datetime', 'Tweet Id','Location'])
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
    # Define working path and filename
    path = os.getcwd()
    filename = path + '/data/' + username + '.csv'
    # Store dataframe in csv with creation date timestamp
    tweets_df1.to_csv(filename, index = False)

AgCyberGarrett
from:AgCyberGarrett
hg_eric
from:hg_eric
ValarezoSilvia
from:ValarezoSilvia
slskdidl629
from:slskdidl629
ClimateKenya254
from:ClimateKenya254
DaisyLeoncio
from:DaisyLeoncio
ywaiaqualife
from:ywaiaqualife
ZaxxonGalaxian
from:ZaxxonGalaxian
Miriamachieng21
from:Miriamachieng21
MAB1961A
from:MAB1961A
AbuProdhan
from:AbuProdhan
DorcasRutunda
from:DorcasRutunda
dhahabu
from:dhahabu
pwpgarden
from:pwpgarden
thisdaringdan
from:thisdaringdan
DBrothers6
from:DBrothers6
CanGeoEdu
from:CanGeoEdu
efiresilience
from:efiresilience
nksnhle
from:nksnhle
Mataaya1
from:Mataaya1
BinagekarAnusha
from:BinagekarAnusha
mnarbs
from:mnarbs
K178aa
from:K178aa
Bornscorer
from:Bornscorer
xuVn1OgFGaCpvDQ
from:xuVn1OgFGaCpvDQ
Roy_Okonji
from:Roy_Okonji
Connect4Climate
from:Connect4Climate
Assuzamukunda
from:Assuzamukunda
JudyKosgei
from:JudyKosgei
GEngida56
from:GEngida56
locagro
from:locagro
IvokainKrieg
from:IvokainKrieg
GDSAofficial1
from:GDSAofficial1
KatjaBessonova
from:KatjaBessonova
aitkakaaki